In [45]:
import os
import re
import subprocess
from collections import defaultdict
import numpy as np
import rasterio
from rasterio.merge import merge
from rasterio.warp import reproject, Resampling
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import glob
import json
from tqdm import tqdm

#import geopandas as gpd
import pandas as pd



import sys
sys.path.append('../')

import utils.basics as bsc 
import utils.plotting as pt
import utils.processing as proc

import src.model_loader as md

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
md.build_unet(in_channels=X[0].shape[0], out_channels=Y[0].shape[0])

AttributeError: module 'src.model_loader' has no attribute 'build_unet'

In [10]:
sites, cfg = get_config("baseline")  # experiment config (site arg irrelevant if we loop sites)

X, Y = build_patched_dataset(cfg, sites, patch_size=32)

In [33]:
# split into train/val/test (70/15/15) with your logic
(X_train, Y_train), (X_val, Y_val), (X_test, Y_test) = split_dataset(X, Y)

# Syntax: X_train, X_val, X_test, y_train, y_val, y_test 
train_dataset = S2CanopyHeightDataset(X_train, Y_train)
val_dataset = S2CanopyHeightDataset(X_val, Y_val)
test_dataset = S2CanopyHeightDataset(X_test, Y_test)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'])
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'])


In [31]:
config

{'patch_size': 32,
 'num_bands': 15,
 'batch_size': 64,
 'lr': 0.001,
 'weight_decay': 0.0003,
 'momentum': 0.9,
 'epochs': 250,
 'huber_delta': 1,
 'device': 'mps'}

In [41]:
# TODO: build model depending on cfg["strategy"]
model = build_unet(in_channels=X[0].shape[0], out_channels=len(output_keys))

NameError: name 'build_unet' is not defined

In [ ]:
# TODO: initialize model depending on cfg["strategy"]

    # TODO: assign losses from cfg["losses"]

    # TODO: training loop, log metrics, save model + logs to out_dir